<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week7_Day3_Tutorial_Build_your_first_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &                                 # runs in background
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF
!pip install ollama
!pip install ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'




In [ ]:
pip install ollama

In [ ]:
dataset = []
with open('/content/cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

Loaded 319 entries


In [4]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))


In [5]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')

Added chunk 1/319 to the database
Added chunk 2/319 to the database
Added chunk 3/319 to the database
Added chunk 4/319 to the database
Added chunk 5/319 to the database
Added chunk 6/319 to the database
Added chunk 7/319 to the database
Added chunk 8/319 to the database
Added chunk 9/319 to the database
Added chunk 10/319 to the database
Added chunk 11/319 to the database
Added chunk 12/319 to the database
Added chunk 13/319 to the database
Added chunk 14/319 to the database
Added chunk 15/319 to the database
Added chunk 16/319 to the database
Added chunk 17/319 to the database
Added chunk 18/319 to the database
Added chunk 19/319 to the database
Added chunk 20/319 to the database
Added chunk 21/319 to the database
Added chunk 22/319 to the database
Added chunk 23/319 to the database
Added chunk 24/319 to the database
Added chunk 25/319 to the database
Added chunk 26/319 to the database
Added chunk 27/319 to the database
Added chunk 28/319 to the database
Added chunk 29/319 to the dat

In [6]:
print(len(VECTOR_DB))

319


In [7]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [8]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [13]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

Ask me a question: what the name of a cat?
Retrieved knowledge:
 - (similarity: 0.75) There is no single cat called the panther. The name is commonly applied to the leopard, puma and the jaguar.

 - (similarity: 0.74) The most popular names for female cats in the U.S. are Missy, Misty, Muffin, Patches, Fluffy, Tabitha, Tigger, Pumpkin and Samantha.

 - (similarity: 0.74) Cats respond most readily to names that end in an “ee” sound.



In [14]:

# THIS LINE OF CODE IS CORRECTED
instruction_prompt = f"""You are a helpful chatbot. Use only the following pieces of context to answer
 the question. Don't make up any new information:
 {''.join([f' - {item}' for item in retrieved_knowledge])}"""

print(instruction_prompt)

You are a helpful chatbot. Use only the following pieces of context to answer
 the question. Don't make up any new information:
  - ('There is no single cat called the panther. The name is commonly applied to the leopard, puma and the jaguar.\n', 0.7505431691901008) - ('The most popular names for female cats in the U.S. are Missy, Misty, Muffin, Patches, Fluffy, Tabitha, Tigger, Pumpkin and Samantha.\n', 0.7435715921654183) - ('Cats respond most readily to names that end in an “ee” sound.\n', 0.7363606349292883)


In [15]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
According to the text, some popular names for cats include:

* Missy
* Misty
* Muffin
* Patches
* Fluffy
* Tabitha
* Tigger
* Pumpkin
* Samantha

Which one do you like best?